In [172]:
import re
from math import ceil

import pandas as pd
import requests
from tqdm.autonotebook import tqdm
from time import sleep

In [4]:
TOKEN = ""
VERSION = "5.131"

In [151]:
df = pd.read_csv("vk_bank_list.csv.zip", compression="zip", index_col=0)
df.head()

,bank_name,vk_owner_id,vk_name
id,,,
1,АОЮниКредит Банк,-23516158,unicreditbank
21,ООО«Примтеркомбанк»,-163466915,primterkombank
53,ПАОБанк «АЛЕКСАНДРОВСКИЙ»,-55357221,alexbankspb
67,АКБ«Энергобанк» (АО),-29459652,energobank
101,АО«БКСБанк»,-162316242,bcsbank


In [152]:
def remove_vk_url(match):
    group = match.group()
    return group.split("|")[1][:-1]


# re.sub(r"\[.+\|.+\]", remove_vk_url, text)

In [153]:
posts = []
comments = []

In [166]:
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    params_wall_get = {"access_token": TOKEN, "v": VERSION, "owner_id": row["vk_owner_id"], "count": 1, "offset": 0}
    r = requests.get("https://api.vk.com/method/wall.get", params=params_wall_get)
    if r.status_code == 200 or "error" in r.json().keys():
        num_page = ceil(r.json()["response"]["count"] / 100)
    else:
        print(r.json())
        break
    params_wall_get["count"] = 100
    for i in range(num_page):  # add tqdm
        params_wall_get["offset"] = i * 100
        r = requests.get("https://api.vk.com/method/wall.get", params=params_wall_get)
        if r.status_code == 200 and "error" not in r.json().keys():
            for post in r.json()["response"]["items"]:
                posts.append({"owner_id": post["owner_id"], "id": post["id"]})
        else:
            print(r)

  0%|          | 0/157 [00:00<?, ?it/s]

In [169]:
len(posts)

212836

In [ ]:
for post in tqdm(posts, total=10_000):
    comm_num = ceil(post["comments"]["count"] / 100)
    for i in range(comm_num):
        params = {
            "access_token": TOKEN,
            "v": VERSION,
            "count": 100,
            "offset": i * 100,
            "owner_id": post["owner_id"],
            "post_id": post["id"],
            "start_comment_id": None,
            "sort": "desc",
            "thread_items_count": 10,
        }
        r = requests.get("https://api.vk.com/method/wall.getComments", params=params)
        sleep(0.3)  # https://vk.com/dev/api_requests
        if r.status_code == 200 and "error" not in r.json().keys():
            for comment in r.json()["response"]["items"]:
                if comment["text"] == "":
                    continue
                url = f"https://vk.com/sber?w=wall{comment['owner_id']}_{comment['post_id']}_r{comment['id']}"
                comments.append({"date": comment["date"], "text": comment["text"], "link": url})
                for thread_comment in comment["thread"]["items"]:
                    if thread_comment != "":
                        url = f"https://vk.com/wall{thread_comment['owner_id']}_{thread_comment['post_id']}?reply={thread_comment['id']}&thread={thread_comment['parents_stack'][0]}"
                        comments.append({"date": thread_comment["date"], "text": thread_comment["text"], "link": url})

        else:
            print(r.json())
            sleep(10)

In [174]:
import numpy as np

In [176]:
np.save("posts.pkl", posts)